In [257]:
import heapq
initial =  [["A", "B", "C"], ["D", "E", "F"], ["G", "H", "I"]]
goal = [["H", "E", "F", "A", "C"], ["B", "D"], ["G", "I"]]
goal.sort()
unique_blocks = list(x for l in initial for x in l)
iteration = 0
prq = []
prq2 = []
prq.append(initial)
prq2.append((0,0, initial))
visited = []
visited_moves = []
itera = 0
state_moves_priority=[]
state_moves_priority.append((0,0, initial))

In [258]:
while state_moves_priority:    
    new_state2 = []
    moves = []
    prq2.sort(reverse=True)
    ns = state_moves_priority.pop()
    prq2=[]
    visited.append(ns)
    itera = itera + 1  
    if itera!=1:
        visited_moves.append(ns[1])
    # traversing towers within states
    for j in range(0, len(list(ns[2]))):
        # first item to be placed on the floor
        if len(list(ns[2])[j]) != 1:
            new_state = []
            new_state = [x[:] for x in list(ns[2])]
            popped = new_state[j].pop()
            new_state = new_state + [[list(ns[2])[j][len(list(ns[2])[j]) - 1]]]
            moves.append((popped, 'table'))
            new_state2.append(new_state)
        # first item to be placed on other blocks
        for k in range(0, len(list(ns[2]))):
            if list(ns[2])[k] != list(ns[2])[j]:
                new_state = []
                new_state = [x[:] for x in list(ns[2])]
                popped = new_state[j].pop()
                moves.append((popped, new_state[k][-1]))
                new_state[k].insert(len(new_state[k]), popped)
                new_state = [x for x in new_state if x != []]
                new_state2.append(new_state)
                
    # determining the positions of blocks in goal state
    position_tuple_goal = []
    for i in range(0, len(goal)):
        for j in range(0, len(goal[i])):
            if j == 0 and j + 1 <= len(goal[i]) - 1:
                position_tuple_goal.append(("goal", goal[i][j], "table", goal[i][j + 1]))
            if j != 0 and j + 1 > len(goal[i]) - 1:
                position_tuple_goal.append(("goal", goal[i][j], goal[i][j - 1], "Top"))
            if len(goal[i]) == 1:
                position_tuple_goal.append(("goal", goal[i][j], "table", "Top"))
            if j != 0 and j + 1 <= len(goal[i]) - 1:
                position_tuple_goal.append(("goal", goal[i][j], goal[i][j - 1], goal[i][j + 1]))

    # Determining the positions of blocks in all the generated states
    position_tuple = []
    for i in range(0, len(new_state2)):
        for j in range(0, len(new_state2[i])):
            for k in range(0, len(new_state2[i][j])):
                if k == 0 and k + 1 <= len(new_state2[i][j]) - 1:
                    position_tuple.append((i, new_state2[i][j][k], "table", new_state2[i][j][k + 1]))
                if k != 0 and k + 1 > len(new_state2[i][j]) - 1:
                    position_tuple.append((i, new_state2[i][j][k], new_state2[i][j][k - 1], "Top"))
                if len(new_state2[i][j]) == 1:
                    position_tuple.append((i, new_state2[i][j][k], "table", "Top"))
                if k != 0 and k + 1 <= len(new_state2[i][j]) - 1:
                    position_tuple.append(
                        (i, new_state2[i][j][k], new_state2[i][j][k - 1], new_state2[i][j][k + 1]))

    # Naive heuristics
    scores_tuple = []
    for i in range(0, len(new_state2)):
        heu1 = 0
        heu2 = 0
        for j in range(0, len(position_tuple)):
            if position_tuple[j][0] == i:
                for k in range(0, len(position_tuple_goal)):

                    # Heuristic 1:this heuristic calculates the number of blocks that are currently not in the correct 'position'. We do not count a block if it is currently in the arm of the crane.
                    if position_tuple[j][1] == position_tuple_goal[k][1] and position_tuple[j][2] != \
                            position_tuple_goal[k][2] and position_tuple[j][3] != position_tuple_goal[k][3] and position_tuple[j][2] != 'table' :
                        heu1 = +1
                     # Heuristic 2:his heuristic is similar to Heuristic 1. It calculates the difference between the current state and the goal state, but looks at the details of each block. If Block A in the goal state is supposed to be on top of Block B and under Block C and in the current state it is neither on top of B or under C, then we add 2 to the heuristic. 
                    if position_tuple[j][1] == position_tuple_goal[k][1] and position_tuple[j][2] != \
                            position_tuple_goal[k][2] and position_tuple[j][3] != position_tuple_goal[k][3] :
                        heu2 = +2    

        scores_tuple.append(heu1 + heu2)


    # Advanced Heuristic
    state_settled = []
    unsettled_blocks = []
    state_moved_once=[]
    state_moved_twice=[]
    #to traverse states
    for i in range(0, len(new_state2)):
        settled_block = []
        moved_once=[]
        moved_twice=[]
        #to traverse towers
        for j in range(0, len(new_state2[i])):
            #to traverse blocks within towers
            for k in range(0, len(new_state2[i][j])):
                #for each blocks within tower, checking for current position(position_tuple) and goal position(position_tuple_goal)
                for l in range(0, len(position_tuple)):
                    if position_tuple[l][0] == i:
                        for m in range(0, len(position_tuple_goal)):
                            if new_state2[i][j][k] == position_tuple[l][1] and position_tuple[l][1] == \
                                    position_tuple_goal[m][1] and position_tuple[l][2] == \
                                    position_tuple_goal[m][2] and position_tuple[l][2] == 'table':

                                settled_block.append(position_tuple[l][1])

                            if new_state2[i][j][k] == position_tuple[l][1] and position_tuple[l][1] == \
                                    position_tuple_goal[m][1] and position_tuple[l][2] == \
                                    position_tuple_goal[m][2] and position_tuple[l][2] in settled_block:

                                settled_block.append(position_tuple[l][1])
                            #heurisitc 4    
                            if new_state2[i][j][k] == position_tuple[l][1] and position_tuple[l][1] == position_tuple_goal[m][1] and (position_tuple[l][2] != position_tuple_goal[m][2] or  position_tuple[l][2] in moved_once):
                                moved_once.append(new_state2[i][j][k])

                            if new_state2[i][j][k] == position_tuple[l][1] and position_tuple[l][1] == position_tuple_goal[m][1] and position_tuple[l][2] == position_tuple_goal[m][2] and  ((position_tuple[l][2] in moved_once) or (position_tuple[l][2] in moved_twice)):
                                moved_twice.append(new_state2[i][j][k])    


        state_settled.append(settled_block)
        state_moved_once.append(moved_once)                                                           
        state_moved_twice.append(moved_twice)                                                           

    for i in range(0, len(state_settled)):
        unsettled_blocks.append(list(set(unique_blocks) - set(state_settled[i])))

    # Determining how many unsettled states are not on top of settled states, i.e. on the table
    unsettled_block_table = []
    for i in range(0, len(unsettled_blocks)):
        unsettled_block_table2 = []
        for j in range(0, len(unsettled_blocks[i])):
            for k in range(0, len(position_tuple)):
                if position_tuple[k][0] == i and position_tuple[k][1] == unsettled_blocks[i][j] and \
                        position_tuple[k][2] != 'table':
                    unsettled_block_table2.append(position_tuple[k][1])
        unsettled_block_table.append(unsettled_block_table2)

        # Final score for all the generated states
    final_score = []
    for i in range(0, len(scores_tuple)):
        final_score.append((scores_tuple[i] +
                           list(len(x) * 2 for x in unsettled_blocks)[i] +
                           list(len(x) * 2 for x in state_moved_once)[i]+
                           list(len(x) * 4 for x in state_moved_twice)[i]+
                           list(len(x) * 1 for x in unsettled_block_table)[i]))
    
    for i in range(0, len(new_state2)):
        prq2.append((final_score[i], new_state2[i]))
        
    state_moves_priority = []
    for i in range(0, len(new_state2)):
        #if (state_priority[i] < min(state_priority)+4) or (itera == 1):
        state_moves_priority.append((final_score[i], moves[i],new_state2[i]))
    
    state_moves_priority.sort(reverse=True)
    #visited_moves.append(state_moves_priority.pop()[1])            
    
    states_achieved = []
    for i in range(0, len(prq2)):
        states_achieved.append(state_moves_priority[i][2])
    
    for i in range(0,len(states_achieved)):
        states_achieved[i].sort()
        
    if goal in states_achieved:
        break

ns = state_moves_priority.pop()
visited_moves.append(ns[1])
print(visited_moves)



[('F', 'table'), ('C', 'table'), ('B', 'table'), ('E', 'table'), ('D', 'B'), ('I', 'table'), ('H', 'table'), ('E', 'H'), ('F', 'E'), ('A', 'F'), ('C', 'A'), ('I', 'G')]
